In [1]:

from itertools import product
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import minimize

import os
os.chdir('..')

from src.swbm import *
from src.plots import *
from src.utils import *

In [2]:
# Load data
input_swbm_ger = pd.read_csv('data/Data_swbm_Germany.csv')
input_swbm_swe = pd.read_csv('data/Data_swbm_Sweden.csv')
input_swbm_esp = pd.read_csv('data/Data_swbm_Spain.csv')

In [3]:
# Values for each variable

cs = [210, 420, 840]
b0 = [0.4, 0.6, 0.8]
g = [0.2, 0.5, 0.8]
a = [2, 4, 8]

# Generate all combinations using itertools.product
all_combinations = list(product(cs, b0, g, a))

In [4]:
# select data and pre-process data
input_swbm_raw = input_swbm_ger.copy()
input_swbm = prepro(input_swbm_raw)


In [7]:
# test
test_combinations = all_combinations[:5]
swbm_param = 'b0'
np.random.seed(42)
results_all = []


for combination in all_combinations:
    # Run SWBM without seasonal variation
    #break
    const_swbm_params = {'c_s': combination[0], 
                         'b0': combination[1], 
                         'g': combination[2], 
                         'a': combination[3]}

    # optimize sinus params for b0
    init_values = [0.5, 2, 5, combination[1]]

    res = minimize(opt_swbm_corr,
                np.asarray(init_values).flatten(),  # has to be 1D
                args=(input_swbm, const_swbm_params, swbm_param),
                options={"maxiter": 500, "disp": False})
    opt_params_df = minimize_res2df(res, [swbm_param])
    
    # Set swbm const_swbm_params
    params_seasonal = {'c_s': combination[0], 
                         'b0': combination[1], 
                         'g': combination[2], 
                         'a': combination[3]}

    # Get sinus curve for current single parameter
    params_seasonal[swbm_param] = seasonal_sinus(
        len(input_swbm),
        amplitude=opt_params_df.loc['amplitude', swbm_param],
        freq=opt_params_df.loc['freq', swbm_param],
        phase=opt_params_df.loc['phase', swbm_param],
        center=opt_params_df.loc['center', swbm_param],
        which=swbm_param
    )

    # Run SWBM with optimized seasonal b0
    preds_seasonal = predict_ts(input_swbm, params_seasonal)
    moists_seasonal, runoffs_seasonal, ets_seasonal, na_count = preds_seasonal

    # calculate correlation
    eval_df = eval_swbm(input_swbm,
                        {'sm': moists_seasonal,
                        'ro': runoffs_seasonal,
                        'le': ets_seasonal},
                        swbm_param)
    
    # save all results in dict 
    combination_dict = {
        'Combination': combination,
        'sum_corr': eval_df['corr'].sum(),
        'SinusParameters': opt_params_df,
        'eval_df': eval_df,
        'na_count': na_count,
        'na_sum': sum(na_count.values())
    }

    results_all.append(combination_dict)

In [8]:
# get dict with highes corr_sum score
max_sum_corr_dict = max(results_all, key=lambda x: x['sum_corr'])
max_sum_corr_dict

{'Combination': (420, 0.4, 0.2, 4),
 'sum_corr': 2.315026471620191,
 'SinusParameters':                  b0
 amplitude  0.149661
 freq       1.987781
 phase      4.908950
 center     0.235192,
 'eval_df':   parameter kind      corr           pval
 0        b0   sm  0.905135   0.000000e+00
 1        b0   ro  0.445194  6.576056e-195
 2        b0   le  0.964697   0.000000e+00,
 'na_count': {'sm': 0, 'le': 0, 'ro': 0},
 'na_sum': 0}